<center>Progetto realizzato da Elena Curti (matr. 185431)

# Ricette
</center>

## Introduzione


## Requisiti
Questo progetto è stato realizzato con Python 3.11 e Neo4j. E' necessaria l'installazione dei seguenti pacchetti:

In [5]:
# %pip install py2neo neo4j-driver pandas

## Operazioni iniziali
Per il corretto funzionamento del progetto proposto, è necessario:
- Creare un nuovo progetto Neo4j 
- Creare all'interno del progetto un DMBS (scegliendo come nome "neo4j", come password "pass" e come versione 5.2.0)
- Aggiungere il plugin APOC.
- Premere su Start.
<!-- 
Da Neo4j creare un nuovo progetto. Scegliere una tra le seguenti alternative:
- Dal progetto appena creato, aggiungere al progetto il file [neo4j.dump](data/neo4j.dump). Creare poi un nuovo DBMS dal file dump (scegliendo come nome "neo4j", come password "pass" e come versione 5.2.0). Aggiungere il plugin APOC. Premere su Start.
- Dal progetto appena creato, aggiungere un DBMS locale (scegliendo come nome, password e versione i valori indicati al punto sopra). Aggiungere il plugin APOC. Premere su Start e poi su Open. Eseguire tutti i comandi riportati nel file [import_commands.txt](data/import_commands.txt).
<br>-->
Eseguire poi le celle di codice, nell'ordine proposto 

## Connessione e autenticazione
Per effettuare la connessione al database, eseguire il seguente script:

In [6]:
from py2neo import Graph
graph = Graph("bolt://localhost:7687",  auth=("neo4j", "pass"))

print("Connessione al database eseguita correttamente!")

Connessione al database eseguita correttamente!


# Schema e dati originali
Lo use case originale è stato trovato sul sito [Recipes Neo4j](https://neo4j.com/graphgists/dd3dedcf-c377-4575-84f4-4d0d30b2a4c5/). Ha la seguente struttura iniziale:
```
    call db.schema.visualization()
```

![schema](img/db_schema2.png)

I nodi Ingredient, Author, Collection, DietType, Keyword hanno un solo campo "name" che ne rappresenta il nome. <br>
Il nodo Recipe ha le seguenti proprietà:
<ul>
    <li>id: campo che identifica univocamente la ricetta </li>
    <li>cookingTime: tempo di cottura </li>
    <li>description: descrizione della ricetta </li>
    <li>name: nome della ricetta </li>
    <li>preparationTime: tempo di preparazione </li>
    <li>skillLevel: difficoltà </li>
</ul>
Gli archi non hanno proprietà. <br><br>

Per importare i dati e aggiungere gli indici ho usato il seguente script. <br>
Lo usecase originale ha previsto i seguenti indici per migliorare la performance delle cypher query:
    <ul>
        <li><i>id</i> su Recipe</li>
        <li><i>name</i> su Ingredient</li>
        <li><i>name</i> su Keyword</li>
        <li><i>name</i> su DietType</li>
        <li><i>name</i> su Author</li>
        <li><i>name</i> su Collection</li>
    </ul>
Il comando ```CREATE INDEX IF NOT EXISTS FOR (n:Recipe) ON (n.id)``` è equivalente a ```CREATE INDEX IF NOT EXISTS ON :Recipe(id)```. Il primo, però, è compatibile con la versione 5.2.0 di Neo4j.

In [7]:
print("Inserimento dei dati in corso... ")

# Ricette
graph.run(""" 
CALL apoc.load.json('https://raw.githubusercontent.com/neo4j-examples/graphgists/master/browser-guides/data/stream_clean.json') YIELD value
WITH value.page.article.id AS id,
       value.page.title AS title,
       value.page.article.description AS description,
       value.page.recipe.cooking_time AS cookingTime,
       value.page.recipe.prep_time AS preparationTime,
       value.page.recipe.skill_level AS skillLevel
MERGE (r:Recipe {id: id})
SET r.cookingTime = cookingTime,
    r.preparationTime = preparationTime,
    r.name = title,
    r.description = description,
    r.skillLevel = skillLevel;
""")
graph.run(""" CREATE INDEX IF NOT EXISTS FOR (n:Recipe) ON (n.id);        // CREATE INDEX IF NOT EXISTS ON :Recipe(id)  """)

# Autori
graph.run("""
//import authors and connect to recipes
CALL apoc.load.json('https://raw.githubusercontent.com/neo4j-examples/graphgists/master/browser-guides/data/stream_clean.json') YIELD value
WITH value.page.article.id AS id,
       value.page.article.author AS author
MERGE (a:Author {name: author})
WITH a,id
MATCH (r:Recipe {id:id})
MERGE (a)-[:WROTE]->(r);
""")
graph.run("""CREATE INDEX IF NOT EXISTS FOR (n:Author) ON (n.name);""" )

# Ingredienti
graph.run("""//import ingredients and connect to recipes
CALL apoc.load.json('https://raw.githubusercontent.com/neo4j-examples/graphgists/master/browser-guides/data/stream_clean.json') YIELD value
WITH value.page.article.id AS id,
       value.page.recipe.ingredients AS ingredients
MATCH (r:Recipe {id:id})
FOREACH (ingredient IN ingredients |
  MERGE (i:Ingredient {name: ingredient})
  MERGE (r)-[:CONTAINS_INGREDIENT]->(i)
);""" )
graph.run("""CREATE INDEX IF NOT EXISTS FOR (n:Ingredient) ON (n.name);""" )

# DietType
graph.run("""//import dietTypes and connect to recipes
CALL apoc.load.json('https://raw.githubusercontent.com/neo4j-examples/graphgists/master/browser-guides/data/stream_clean.json') YIELD value
WITH value.page.article.id AS id,
       value.page.recipe.diet_types AS dietTypes
MATCH (r:Recipe {id:id})
FOREACH (dietType IN dietTypes |
  MERGE (d:DietType {name: dietType})
  MERGE (r)-[:DIET_TYPE]->(d)
);""" )
graph.run("""CREATE INDEX IF NOT EXISTS FOR (n:DietType) ON (n.name);""" )

# Collections
graph.run("""//import collections and connect to recipes
CALL apoc.load.json('https://raw.githubusercontent.com/neo4j-examples/graphgists/master/browser-guides/data/stream_clean.json') YIELD value
WITH value.page.article.id AS id,
       value.page.recipe.collections AS collections
MATCH (r:Recipe {id:id})
FOREACH (collection IN collections |
  MERGE (c:Collection {name: collection})
  MERGE (r)-[:COLLECTION]->(c)
);""" )
graph.run("""CREATE INDEX IF NOT EXISTS FOR (n:Collection) ON (n.name);""" )

# Keyword
cq = """//import keywords and connect to recipes
CALL apoc.load.json('https://raw.githubusercontent.com/neo4j-examples/graphgists/master/browser-guides/data/stream_clean.json') YIELD value
WITH value.page.article.id AS id,
       value.page.recipe.keywords AS keywords
MATCH (r:Recipe {id:id})
FOREACH (keyword IN keywords |
  MERGE (k:Keyword {name: keyword})
  MERGE (r)-[:KEYWORD]->(k)
);"""
# graph.run(cq)
cq = """CREATE INDEX IF NOT EXISTS FOR (n:Keyword) ON (n.name);""" 
# graph.run(cq)


print("Tutti i dati sono stati importati correttamente!")

Inserimento dei dati in corso... 
Tutti i dati sono stati importati correttamente!


## Modifiche effettuate
Per arricchire il progetto ho deciso di aggiungere dei dati presi da Kaggle e di eseguire alcune modifiche allo schema originale. 
<br>
<mark>TODOaggiunti altri campi dal file originale  <br>



In [8]:
# Definisco un'eccezione usata nel seguito
class MiaStopExecution(Exception):
    """Il raise di questa classe provoca l'interruzione dell'esecizione della cella, senza interrompere il kernel"""
    def _render_traceback_(self):
        pass

# Prima di effettuare le modifiche, copio i file necessari nella cartella "import" del DBMS, mediante il seguente script:
import os 
import shutil
DATA_FOLDER ="data"+os.path.sep

cq ="""CALL dbms.listConfig() YIELD name, value
    WHERE name="server.directories.import"
    RETURN value"""
DIRECTORY_IMPORT = graph.run(cq).evaluate()
print("Cartella import del DMBS: " + DIRECTORY_IMPORT)

def copy_in_import_dir(file_name):
    source = DATA_FOLDER + file_name
    destination = DIRECTORY_IMPORT + os.path.sep + file_name
    shutil.copy(source, destination)

copy_in_import_dir("FoodData.csv") 
copy_in_import_dir("gz_recipe.csv") 


Cartella import del DMBS: D:\Elena\_Elena\Shared\Universita\Magistrale\Big_data\Anno_22_23\TO_DEL_QUANDO_ESAME_DATO\Neo4J\relate-data\dbmss\dbms-b3e0a01c-d1be-4257-8d27-97054bc0e187\import


### 1. Rimozione di Keyword
Ho deciso di rimuovere il nodo Keyword. Ho quindi evitato di eseguire le ultime due cypher query nello script precedente. Alternativamente, avrei potuto anche usare il seguente comando: 

In [9]:
graph.run("MATCH (n:Keyword) DETACH DELETE n")

(No data)

### 2. Aggiunta di altre ricette
Per arricchire il dataset, ho deciso di inserire altre ricette al database. Ho quindi scaricato il file [gz_recipe.csv](data/gz_recipe.csv) da https://www.kaggle.com/datasets/edoardoscarpaci/italian-food-recipes, contenente delle ricette estratte dal sito [GialloZafferano](https://www.giallozafferano.it/). <br>
Il file scaricato contiene i seguenti campi:
- id: id che identifica univocamente la ricetta (intero da 0 a 5938).
- Nome: nome del piatto
- Categoria: categoria della ricetta (Primi, Secondi, ...)
- Link: link della ricetta
- Persone/Pezzi: numero di persone o pezzi della ricetta
- Ingredienti: lista di ingredienti e relative quantità. Ad esempio: [['Mascarpone', '750g'], ['Uova', '260g']]
- Steps: contenuto della ricetta

Per i inserire questi dati ho deciso di:
- Aggiungere a Recipe le proprietà 
    - "fonte" che avrà il valore "GialloZafferano" o "BBC GoodFood"
    - "persone_pezzi" per indicare il numero di persone o pezzi
- Aggiungere a CONTAINS_INGREDIENTS una proprietà "quantita", contenente la quantità di un ingrediente in una ricetta

Come precedentemente spiegato, il campo id di Recipe è univoco nelle ricette già inserite. Aggiungendo i dati da un altra sorgente, si potrebbero avere dei record con lo stesso id. Prima di inserire le nuove ricette, quindi, controllo che gli id delle nuove ricette siano tutti diversi da quelli delle ricette già inserite. In questo caso, per fortuna, non ci sono conflitti sugli id e si può procedere con gli inserimenti. 

In [10]:
# Aggiungo le ricette di GialloZafferano. 

# Controllo che non ci siano già ricette con gli id
cq="""
LOAD CSV WITH HEADERS FROM 'file:///gz_recipe.csv' AS value
MATCH (r:Recipe {id:value["id"]})
WHERE r.fonte IS NULL OR r.fonte<>"GialloZafferano"
return count(r)
"""
if graph.run(cq).evaluate() != 0:
    print("Le ricette di BBC e quelle di GialloZafferano hanno degli id in comune!")
    raise(MiaStopExecution)

# Inserisco le ricette
cq="""
LOAD CSV WITH HEADERS FROM 'file:///gz_recipe.csv' AS value
MERGE (r:Recipe {id: value["id"]})
SET r.name = value["Nome"],
    r.description = value["Steps"],
    r.persone_pezzi = value["Persone/Pezzi"],
    r.fonte = "GialloZafferano"
"""
graph.run(cq)

cq="""MATCH (r:Recipe) WHERE r.fonte IS NULL SET r.fonte="BBC GoodFood" """
graph.run(cq)

# Inserisco gli ingredienti
cq="""
LOAD CSV WITH HEADERS FROM 'file:///gz_recipe.csv' AS value
WITH value["id"] AS id, replace(value["Ingredienti"], '"', "") AS ris1
WITH id, replace(ris1, "[[", "") AS ris2
WITH id,replace(ris2, "]]", "") AS ris3
WITH id,replace(ris3, "'", "") AS ris4
WITH id,split(ris4, "], [") AS ingrs_list
MATCH (r:Recipe {id:id})
WHERE ingrs_list[0]<>"[]"   // Alcune ricette non hanno ingredienti nè descrizione (es. Churros red velvet)
FOREACH (ingr_quantita_string IN ingrs_list |
    MERGE (i:Ingredient {name: split(ingr_quantita_string, ", ")[0]})
    MERGE (r)-[:CONTAINS_INGREDIENT {quantita: split(ingr_quantita_string, ", ")[1]}]->(i)
)
""" 
graph.run(cq)


# Creo le categorie 
cq="""
LOAD CSV WITH HEADERS FROM 'file:///gz_recipe.csv' AS value
MATCH (r:Recipe {id:value["id"]})
WHERE value["Categoria"] IS NOT NULL
MERGE (c:Collection {name: value["Categoria"]})
MERGE (r)-[:COLLECTION]->(c)
"""
graph.run(cq)



(No data)

### 3. Inserimento degli allergeni
Ho aggiunto un campo "is_allergene" al nodo Ingredient. Ho scaricato da Kaggle due file:
- [allergens.json](data/allergens.json) (https://www.kaggle.com/datasets/elenacivati/allergensjson): contenente un elenco non molto ampio di allergeni in italiano, inglese e altre lingue.
- [FoodData.csv](data/FoodData.csv) (https://www.kaggle.com/datasets/boltcutters/food-allergens-and-allergies): contenente un elenco più ampio ma solo in inglese

Ho quindi impostato gli ingredienti contenuti nei file con is_allergene=True. Ho invece settato is_allergene=False per i restanti ingredienti. 

In [11]:
# Prendo dal file allergens.json i nomi degli ingredienti in inglese e italiano. 
# Converto poi in csv, in modo da semplificare la lettura del file successivamente
import pandas as pd
df_allergens = pd.read_json(DATA_FOLDER + "allergens.json").transpose()
df_allergens["en"]=df_allergens.index
df_allergens["en"]=df_allergens["en"].apply(lambda x: x[3:])
df_allergens["it"]=df_allergens["name"].apply(lambda x: x["it"])
df_allergens=df_allergens[["en", "it"]]
df_allergens.to_csv(DATA_FOLDER + "allergens.csv", index=False)
copy_in_import_dir("allergens.csv")

# Setto gli allergeni
cq="""
LOAD CSV WITH HEADERS FROM 'file:///FoodData.csv' AS value
LOAD CSV WITH HEADERS FROM 'file:///allergens.csv' AS value2
MATCH (i:Ingredient)
WHERE 
    toLower(i.name) CONTAINS toLower(value["Food"])  OR 
    toLower(i.name) CONTAINS toLower(value2["en"])  OR 
    toLower(i.name) CONTAINS toLower(value2["it"])  
SET i.is_allergene=True;"""
graph.run(cq)

# Setto i restanti ingredienti come non allergeni
cq="""
MATCH (i:Ingredient)
WHERE i.is_allergene IS NULL
SET i.is_allergene=False; """
graph.run(cq)


(No data)

### 4. Nuovo indice
Per velocizzare in futuro la ricerca dei nomi delle ricette, ho deciso di inserire un nuovo indice sulla proprietà name di Recipe.

In [12]:
graph.run(" CREATE INDEX IF NOT EXISTS FOR (n:Recipe) ON (n.name);")

(No data)

### Schema finale
Dopo le modifiche, lo schema finale è il seguente:

![schema](img/db_schema_finale.png)

In [13]:
def print_full_table(query):
    g_run = graph.run(query)
    # Stampo l'intestazione
    intestazione=g_run.keys()
    [print(f"{i:25}", end="") for i in intestazione]
    print()
    [print("-", end="") for _ in range(25) for _ in range(len(intestazione))]
    print()

    # Stampo i dati
    data = g_run.data()
    for d in data: 
        for k,v in d.items():
            print(f"{str(v):25}", end="")
        print()

print_full_table("CALL db.schema.nodeTypeProperties")

nodeType                 nodeLabels               propertyName             propertyTypes            mandatory                
-----------------------------------------------------------------------------------------------------------------------------
:`Author`                ['Author']               name                     ['String']               True                     
:`Ingredient`            ['Ingredient']           name                     ['String']               True                     
:`Ingredient`            ['Ingredient']           is_allergene             ['Boolean']              True                     
:`DietType`              ['DietType']             name                     ['String']               True                     
:`Collection`            ['Collection']           name                     ['String']               True                     
:`Recipe`                ['Recipe']               id                       ['String']               True              

In [14]:
print_full_table("CALL db.schema.relTypeProperties")

relType                  propertyName             propertyTypes            mandatory                
----------------------------------------------------------------------------------------------------
:`WROTE`                 None                     None                     False                    
:`CONTAINS_INGREDIENT`   quantita                 ['String']               False                    
:`DIET_TYPE`             None                     None                     False                    
:`COLLECTION`            None                     None                     False                    


## Interrogazioni
- Cerca le ricette con due ingredienti scelti
- Cerca una ricetta in base al nome 
- Ricetta più veloce da preparare
- Cerca lo chef che ha scritto più ricette
- Stampa le ricette senza allergeni
- Partendo da una ricetta, mostra i suggerimenti (collection, diet type, autore, skilLevel, fonte)
- Rimettere Keyword e "The Top 5 tags co-occurring with keyword ..., with respective frequency count"



In [42]:
def print_recipes(recipe_list, short_print="y"):
    print("\nRicette trovate: ")
    ricetta_trovata = False
    cont = 0
    for r in recipe_list:
        ricetta_trovata = True
        # if short_print=='-':
        #     short_print = input("Stampare anche i dettagli della/e ricetta/e (y/n)? ") 
        import pprint
        pprint.pprint(r.items())
        node_recipe = r[r.keys()[0]]

        if short_print == 'y':
            node_recipe = r[r.keys()[0]]
            if dict(node_recipe).__contains__("name"):
                print("\t", node_recipe["name"])
                continue
        
        print("------ Ricetta numero", node_recipe.get("id", ""), "------")
        if dict(node_recipe).__contains__("name"):
            print("name:", node_recipe["name"])
        for k,v in node_recipe.items():
            if k!="name":
                print(k, "\b:", v)

        # TODO aggiungere altri campi (ingrediente, autore, collection, diettype)

        cont += 1
        if cont == 3:
            if input("Stampare altre 3 ricette (y/n)?")=='y':
                cont = 0
            else:
                break
        
    if not ricetta_trovata:
        print("------Nessuna ricetta!------")

    
def print_ingredients(ingr_list):
    trovato = False
    for r in ingr_list:
        trovato = True
        node_ingr = r[r.keys()[0]]
        print("\t",node_ingr["name"])
        
    if not trovato:
        print("------Nessun ingrediente!------")




### Stampa una ricetta con due ingredienti
Il seguente codice stampa una ricetta che abbia almeno due ingredienti scelti dall'utente.

In [27]:
# Cerco le ricette con un ingrediente
print("************ Stampa una ricetta con due ingredienti ************ ")

print("Qualche suggerimento per la scelta degli ingredienti. Ecco 5 ingredienti casuali: ")
cq = """ 
MATCH (i:Ingredient)
WITH i, rand() as num
RETURN i.name AS name, i.is_allergene AS is_allergene
ORDER BY num
LIMIT 5
"""
for i in graph.run(cq):
    print("\t", i["name"])
print()

# Chiedo il primo ingrediente e stampo gli ingredienti più usati con esso
ingrediente1=""
while ingrediente1=="":
    ingrediente1=input("Inserisci primo ingrediente: ")
# ingrediente1="mascarpone"

print("Ecco 5 ingredienti usati spesso con " +ingrediente1)
cq = """ 
MATCH (i_scelto:Ingredient)<-[:CONTAINS_INGREDIENT]-(:Recipe)-[:CONTAINS_INGREDIENT]->(i_raccomandato:Ingredient)
WHERE toLower(i_scelto.name) CONTAINS toLower('"""+ingrediente1+"""')
WITH i_raccomandato,  COUNT(i_raccomandato) AS num
RETURN i_raccomandato, num
ORDER BY num DESC
LIMIT 5
"""
print_ingredients(graph.run(cq))

ingrediente2=""
while ingrediente2=="": 
    ingrediente2=input("Inserisci secondo ingrediente: ")
# ingrediente2="Zucchero"

cq = """ 
MATCH (i1:Ingredient)<-[:CONTAINS_INGREDIENT]-(r:Recipe)-[:CONTAINS_INGREDIENT]->(i2:Ingredient)
WHERE 
    toLower(i1.name) CONTAINS toLower('"""+ingrediente1+"""') AND 
    toLower(i2.name) CONTAINS toLower('"""+ingrediente2+"""') 
RETURN r
"""
# for i in graph.run(cq):
#     print(i["r.name"])
print_recipes(graph.run(cq))

************ Stampa una ricetta con due ingredienti ************ 
Qualche suggerimento per la scelta degli ingredienti. Ecco 5 ingredienti casuali: 
	 wholegrain basmati rice
	 sprinkles and stars
	 silver pearl balls
	 Sciroppo di latte di mandorla
	 Ortiche

Ecco 5 ingredienti usati spesso con wholegrain basmati rice
	 garlic clove
	 onion
	 ground cumin
	 coriander
	 flaked almond

Ricette trovate: 
	 Bone broth
	 Turkish pilaf with saffron & goji berries


### Stampa i dettagli di una ricetta
Quando l'utente ha trovato la ricetta desiderata, può decidere di visionarne i dettagli con il seguente script:

In [160]:
nome_ricetta="Bone broth"
while nome_ricetta=="": 
    nome_ricetta=input("Inserisci nome della ricetta: ")


# cq = "MATCH (a:Author)-[:WROTE]->(r:Recipe {name: '"+nome_ricetta+"""'})  
cq = """  
MATCH (r:Recipe)
WHERE r.name='Tiramisù' OR r.name='Bone broth'
OPTIONAL MATCH (a:Author)-[:WROTE]->(r)
OPTIONAL MATCH (c:Collection)<-[:COLLECTION]-(r)
OPTIONAL MATCH (d:DietType)<-[:DIET_TYPE]-(r)
OPTIONAL MATCH (i:Ingredient)<-[r_i:CONTAINS_INGREDIENT]-(r)
RETURN r AS ricetta, COLLECT(DISTINCT c) AS categorie, COLLECT(DISTINCT i) AS ingredienti, COLLECT(DISTINCT d) AS tipi_dieta, COLLECT(DISTINCT r_i) AS quantita_ingredienti
"""


list_recipes = graph.run(cq)
# list_recipes.sample_size = len(graph.run(cq).data())

# display(list_recipes)#["num_results"])

for res in list_recipes:
    tmp = dict(res)
    ricetta = tmp["ricetta"]
    print("---------------")
    for k,v in ricetta.items():
        print(k, "\b:",v)
    
    if tmp.__contains__("categorie"):
        categorie = tmp["categorie"]
        print("Categorie:", [v for c in categorie for _,v in c.items()])
    

    if tmp.__contains__("ingredienti"):
    
        ingredienti =  [c["name"] + ("" if c["is_allergene"] else " (Allergene!)") for c in tmp["ingredienti"]]
    
    # print("Ingredienti:", [c["name"] + ("" if c["is_allergene"] else " (Allergene!)") for c in ingredienti])
    if tmp.__contains__("quantita_ingredienti"):
    
        quantita_ingredienti = [dict(q_i).get("quantita", "") for q_i in tmp["quantita_ingredienti"]]
        # print(len(ingredienti), len(quantita_ingredienti))

        res_ingr = []
        for i in range(len(ingredienti)):
            res_ingr.append( ingredienti[i] + " " + quantita_ingredienti[i])
    else:
        res_ingr = ingredienti
    print("Ingredienti:", res_ingr)


    # import pprint
    # pprint.pprint(tmp)
    # print()

# if list_recipes.data() == []:
#     print("--- Nessuna ricetta trovata --- ")
# else:
# for r in list_recipes:
# # if short_print=='-':
# #     short_print = input("Stampare anche i dettagli della/e ricetta/e (y/n)? ") 
#     import pprint
#     pprint.pprint(r.items())
#     # node_recipe = r[r.keys()[0]]

---------------
persone_pezzi: 8
fonte: GialloZafferano
name: Tiramisù
description: Per preparare il tiramisù preparate il caffé con la moka per ottenerne 300 g, poi zuccherate a piacere (noi abbiamo messo un cucchiaino) e lasciatelo raffreddare in una ciotolina bassa e ampia. Separate le uova dividendo gli albumi dai tuorli ricordando che per montare bene gli albumi non dovranno presentare alcuna traccia di tuorlo. Montate i tuorli con le fruste elettriche, versando solo metà dose di zucchero Non appena il composto sarà diventato chiaro e spumoso, e con le fruste ancora in funzione, potrete aggiungere il mascarpone, poco alla volta Incorporato tutto il formaggio avrete ottenuto una crema densa e compatta Quando saranno schiumosi versate il restante zucchero un po’ alla volta Dovrete montarli a neve ben ferma così stempererete il composto. Dopodiché procedete ad aggiungere la restante parte di albumi, poco alla volta mescolando molto delicatamente dal basso verso l'alto La crema al mas

## Inserimenti


## Modifiche

## Cancellazioni
